In [1]:
# Let us use the classes from the Graph Theory Notebook

In [2]:
import string

class Node(object):
    '''
    This is a class that represents a node. The functions of a class are called methods. Instantiations of this class
    will create objects that have access to the methods. 
    
    Each method begins with an argument of "self" which lets the methos know which object it belongs to.
    
    For the Node object, we care about the name of the node and a dictionary naming the adjacent nodes. The adjacent
    dictionary keys will be the name's of adjacent nodes and the value will be the weight of that edge.
    '''
    def __init__(self, name):
        # The init method is magic python method that get's called when an object is instantiated. 
        self.name = name  # Take the name given at instantiation and assign it to the object
        self.adjacent = {}  # When we create the node, we don't know about any adjacent nodes. Create a placeholder
        
    def __str__(self):
        # When printing a node, format it like this:
        return "Node {}: {}".format(self.name, self.adjacent)
    
    def __repr__(self):
        # When representing this object, use the str method
        return self.__str__()
        

In [6]:
class Graph(object):
    '''
    This class represents a graph. The graph is consists of a collection of nodes. 
    '''
    def __init__(self, adjacency_matrix=None):
        '''
        When the graph is instantiated it will not know about any of its member nodes. There is an optional 
        `adjacency_matrix` argument that defaults to None. If an adjacency_matrix is provided, then we can 
        create the member nodes, their relationships, and add them to this graph.
        '''
        self.nodes = {}
        if adjacency_matrix:
            # We were provided an adjacency matrix
            self._create_member_nodes_from_adjacency_matrix(adjacency_matrix)
            
    def __str__(self):
        return '\n'.join([str(n) for n in self.nodes.values()])
    
    def __repr__(self):
        return self.__str__()
    
    def _create_member_nodes_from_adjacency_matrix(self, adjacency_matrix):
        # A preceeding underscore indicetes a method that is only called within the class. 
        # Create nodes and weights from an adjacency matrix (list of lists) and assign them to this graph
        letters = list(string.ascii_lowercase)  # Get a list of letters to assign to the nodes we create
        needed_letters = len(adjacency_matrix)
        if needed_letters > len(letters):
            # Give the user an error that the input matrix is too large to have 
            raise ValueError("The matrix is too large to create from 26 letters. Please manually assign names to nodes and then add them to the graph.")
        
        for i, row in enumerate(adjacency_matrix):  # get an index of the row and the contentx of the row
            letter_name = letters[i]
            node = Node(letter_name)
            for j, column_value in enumerate(row):
                if column_value != 0:  # There is an adjacent node here
                    adjacent_letter_name = letters[j]
                    node.adjacent[adjacent_letter_name] = column_value
            self.nodes[letter_name] = node
            
    def get_distance_of_path(self, path):
        # Given a list of node names within the graph, find its distance using the node adjacency measures
        distance = 0
        for i, node_name in enumerate(path[:-1]):
            # Gets the index and node name for each in the path except for the last one
            this_node = self.nodes[node_name]
            next_node = self.nodes[path[i+1]]
            distance += this_node.adjacent[next_node.name]  # Get the distance from this_node to next node
        
        return distance
    
    def find_all_paths_between(self, a, b, path=None):
        if not path:
            # No path list provided, create one
            path = []

        path = path + [a] # Add a to the path

        if a == b:  # This is a path to itself, at the end of a path
            return [path]
        if a not in self.nodes:  # a could not be found in the graph
            return []
        paths = []
        for node in self.nodes[a].adjacent:  # look to adjacent nodes of a
            if node not in path:
                newpaths = self.find_all_paths_between(node, b, path)  # Recursivly go down another level
                for newpath in newpaths:
                    paths.append(newpath)
        return paths
    
    def shortest_path(self, a, b):
        # Get the distance of each weighted path and return the shortest
        all_paths = self.find_all_paths_between(a, b)
        
        if not all_paths:
            return []
        
        # The shortest path will be the first path in the list
        shortest_path = None
        shortest_distance = None
        
        for path in all_paths:
            p_distance = self.get_distance_of_path(path)
            if not shortest_distance or p_distance < shortest_distance:
                # This path is shorter
                shortest_path = path
                shortest_distance = p_distance
                
        return shortest_path
    
    def is_tree(self):
        # returns True if the graph is a tree, otherwise False
        
        node_names = list(self.nodes.keys())
        first = node_names[0]  # Make sure the first node has a path to all other nodes
        second = node_names[1]  # Used to determine if acyclic
        
        for node in node_names:
            if node == first:
                continue  # Don't compare it against itself
            if not self.find_all_paths_between(first, node):
                return False  # No path was found between first and node, this must not be a tree
            
        # Check if cyclic. Trees must be acyclic.
        paths_back = self.find_all_paths_between(first, second)
        adjacent = [first, second] in paths_back
        for p in paths_back:
            if set(p) == set(node_names) and adjacent:
                # This path contains each node and returns to itself. It is cyclic
                return False
        
        return True
        

In [15]:
# 3
matrix = [
    # Distance to 
    # A   B   C   D   E   F   G   H   I   J   K   L
    [ 0,  9,  0,  14, 0,  0,  0,  0,  0,  0,  0,  0],  # Node A distances to [A, B, C, D, E, F, G, G, I, J, K]
    [ 9,  0,  13, 0,  12, 0,  0,  0,  0,  0,  0,  0],  # B
    [ 0,  13, 0,  0,  0,  24, 0,  0,  0,  0,  0,  0],  # C
    [ 14, 0,  0,  0,  10, 0,  8,  0,  0,  0,  0,  0],  # D
    [ 0,  12, 0,  10, 0,  30, 0,  20, 0,  0,  0,  0],  # E
    [ 0,  0,  24, 0,  30, 0,  0,  0,  23, 0,  0,  0],  # F
    [ 0,  0,  0,  8,  0,  0,  0,  29, 0,  22, 0,  0],  # G
    [ 0,  0,  0,  0,  20, 0,  29, 0,  2,  0,  15, 0],  # H
    [ 0,  0,  0,  0,  0,  23, 0,  2,  0,  0,  0,  1],  # I
    [ 0,  0,  0,  0,  0,  0,  22, 0,  0,  0,  4,  0],  # J
    [ 0,  0,  0,  0,  0,  0,  0,  15, 0,  4,  0,  17],  # K
    [ 0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  17, 0],  # L
] 

g = Graph(matrix)
g.shortest_path('a', 'l')

In [22]:
# 4

# This time we instantiate the Node and Graph without the matrix
a, b, c, d, e = Node('a'), Node('b'), Node('c'), Node('d'), Node('e')
a.adjacent = {'c': 29, 'd': 8, 'e': 11}
b.adjacent = {'c': 4, 'd':19, 'e': 13}
c.adjacent = {'a': 29, 'b': 4, 'd': 16, 'e': 3}
d.adjacent = {'a': 8, 'b': 19, 'c': 16, 'e': 14}
e.adjacent = {'a': 11, 'b': 13, 'c': 3, 'd': 14}

g = Graph()
g.nodes = {n.name: n for n in [a, b, c, d, e]}

path = g.shortest_path('a', 'b')
distance = g.get_distance_of_path(path)

print(f"The shortest way from A -> B is via {path} with a distance of {distance}")

The shortest way from A -> B is via ['a', 'e', 'c', 'b'] with a distance of 18
